In [ ]:
import os
import re
import numpy as np

# Define the base directory where the expansible log directories are located
for method in ["oneline_st_an_sd", "trafficstream", "stkec", "eac"]:  # 
    print(" " * 50 + "*" * 10 + method +"*" * 10)
    # method = "stkec"  # expansible / retrained / fullretrained / static / trafficStream / stkec
    base_dir = f"./log/SD/{method}"

    # Define the directories to parse
    directories = [f"{base_dir}-42"]

    # Define a regular expression to extract the metric values
    metric_pattern = re.compile(r"(\d+|Avg)\s+(MAE|RMSE|MAPE)\s+([\d\.\s]+)")
    total_time_pattern = re.compile(r"total time:\s+([\d\.]+)")

    # Initialize a dictionary to store metrics
    metrics = {
        "3": {"MAE": [], "RMSE": [], "MAPE": []},
        "6": {"MAE": [], "RMSE": [], "MAPE": []},
        "12": {"MAE": [], "RMSE": [], "MAPE": []},
        "Avg": {"MAE": [], "RMSE": [], "MAPE": []}
    }

    # List to store total times
    total_times = []

    # Function to parse the log file and extract the last column value of metric lines
    def parse_log_file(file_path):
        with open(file_path, "r") as file:
            if method == 'pretrain_st_pems':
                lines = file.readlines()[-15:]
            else:
                lines = file.readlines()[-22:]
            for line in lines:
                match = metric_pattern.search(line)
                
                total_time_match = total_time_pattern.search(line)
                if match:
                    key, metric_type, values = match.groups()
                    values_list = list(map(float, values.split()))
                    last_value = values_list[-1]  # Extract the last value
                    if key in metrics:
                        metrics[key][metric_type].append(last_value)
                if total_time_match:
                    total_time = float(total_time_match.group(1))
                    total_times.append(total_time)


    # Process each directory and extract metrics
    for directory in directories:
        log_file_path = os.path.join(directory, f"{method}.log")
        if os.path.exists(log_file_path):
            parse_log_file(log_file_path)


    # Function to compute mean and variance for the metrics
    def compute_statistics(metric_data):
        metric_array = np.array(metric_data)
        mean = np.mean(metric_array)
        variance = np.std(metric_array)
        return mean, variance

    formatted_results = []

    for key in metrics:
        for metric_type in ["MAE", "RMSE", "MAPE"]:
            mean, variance = compute_statistics(metrics[key][metric_type])
            formatted_results.append(f"{mean:.2f}±{variance:.2f}\t")

    # Compute mean for total times
    time_mean, time_variance = compute_statistics(total_times)

    # Output the formatted results
    header = " "*22 + "3" + " " * 48 + "6" + " " * 46 + "12" + " " * 46 + "Avg" + " " * 27 + "Total Time"
    sub_header = "    {:<10}\t     {:<10}\t     {:<10}\t     {:<10}\t     {:<10}\t     {:<10}\t     {:<10}\t     {:<10}\t     {:<10}\t     {:<10}\t     {:<10}\t     {:<10}".format(
        "MAE", "RMSE", "MAPE", "MAE", "RMSE", "MAPE", "MAE", "RMSE", "MAPE", "MAE", "RMSE", "MAPE"
    )
    values_row = "{:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10}  {:.2f}±{:.2f}".format(*formatted_results, time_mean,time_variance)

    print(header)
    print(sub_header)
    print(values_row)
    print('\n')

                                                  **********oneline_st_an_sd**********
                      3                                                6                                              12                                              Avg                           Total Time
    MAE       	     RMSE      	     MAPE      	     MAE       	     RMSE      	     MAPE      	     MAE       	     RMSE      	     MAPE      	     MAE       	     RMSE      	     MAPE      
nan±nan	   nan±nan	   nan±nan	   nan±nan	   nan±nan	   nan±nan	   nan±nan	   nan±nan	   nan±nan	   nan±nan	   nan±nan	   nan±nan	    nan±nan


                                                  **********trafficstream**********
                      3                                                6                                              12                                              Avg                           Total Time
    MAE       	     RMSE      	     MAPE      	     MAE       	     RMSE      	  

/home/bd2/miniconda3/envs/ANATS/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/bd2/miniconda3/envs/ANATS/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/bd2/miniconda3/envs/ANATS/lib/python3.12/site-packages/numpy/_core/_methods.py:223: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/bd2/miniconda3/envs/ANATS/lib/python3.12/site-packages/numpy/_core/_methods.py:181: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/home/bd2/miniconda3/envs/ANATS/lib/python3.12/site-packages/numpy/_core/_methods.py:215: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [5]:
import os
import re
import numpy as np

# 初始化一个字典来存储所有方法的结果
all_results = {}

# 首先收集所有方法的数据
for method in ["retrain_st_sd","pretrain_st_sd","online_st_nn_sd", "online_st_an_sd", "trafficstream", "stkec", "eac", "pecpm","dlf","team"]:
    base_dir = f"./log/GBA/{method}"
    directories = [f"{base_dir}-42"]
    
    metric_pattern = re.compile(r"(\d+|Avg)\s+(MAE|RMSE|MAPE)\s+([\d\.\s]+)")
    total_time_pattern = re.compile(r"total time:\s+([\d\.]+)")
    
    metrics = {
        "3": {"MAE": [], "RMSE": [], "MAPE": []},
        "6": {"MAE": [], "RMSE": [], "MAPE": []},
        "12": {"MAE": [], "RMSE": [], "MAPE": []},
        "Avg": {"MAE": [], "RMSE": [], "MAPE": []}
    }
    
    total_times = []
    
    def parse_log_file(file_path):
        with open(file_path, "r") as file:
            lines = file.readlines()[-22:] if method != 'pretrain_st_pems' else file.readlines()[-15:]
            for line in lines:
                match = metric_pattern.search(line)
                total_time_match = total_time_pattern.search(line)
                if match:
                    key, metric_type, values = match.groups()
                    values_list = list(map(float, values.split()))
                    if key in metrics:
                        metrics[key][metric_type].append(values_list[-1])
                if total_time_match:
                    total_times.append(float(total_time_match.group(1)))
    
    for directory in directories:
        log_file_path = os.path.join(directory, f"{method}.log")
        if os.path.exists(log_file_path):
            parse_log_file(log_file_path)
    
    def compute_statistics(metric_data):
        metric_array = np.array(metric_data)
        return np.mean(metric_array), np.std(metric_array)
    
    # 存储每个方法的结果
    method_results = []
    for key in ["3", "6", "12", "Avg"]:
        for metric_type in ["MAE", "RMSE", "MAPE"]:
            mean, std = compute_statistics(metrics[key][metric_type])
            method_results.append((mean, std))
    
    time_mean, time_std = compute_statistics(total_times)
    method_results.append((time_mean, time_std))
    
    all_results[method] = method_results

# 生成一个大的LaTeX表格
print("\\begin{table*}[htbp]")
print("\\centering")
print("\\small")  # 使用小号字体使表格更紧凑
print("\\setlength{\\tabcolsep}{4pt}")  # 减小列间距
print("\\begin{tabular}{l|ccc|ccc|ccc|ccc}")
print("\\toprule")  # 使用 booktabs 包的粗线
print("\\multirow{2}{*}{Method} & \\multicolumn{3}{c|}{3\#Step} & \\multicolumn{3}{c|}{6\#Step} & \\multicolumn{3}{c|}{12\#Step} & \\multicolumn{3}{c}{Avg} \\\\")
print("\\cmidrule(lr){2-4} \\cmidrule(lr){5-7} \\cmidrule(lr){8-10} \\cmidrule(lr){11-13}")  # 使用 booktabs 的中等粗细线
print("& MAE & RMSE & MAPE & MAE & RMSE & MAPE & MAE & RMSE & MAPE & MAE & RMSE & MAPE \\\\")
print("\\midrule")  # 使用 booktabs 的中等粗细线

# 为每个方法生成数据行
for method, results in all_results.items():
    row_data = [f"{mean:.2f}" for mean,_ in results[:-1]]  # 去掉最后一个时间数据
    print(f"{method.replace('_', '-')} & {' & '.join(row_data)} \\\\")
    print("\\midrule")  # 使用 booktabs 的中等粗细线

print("\\bottomrule")  # 使用 booktabs 的粗线
print("\\end{tabular}")
print("\\caption{Experimental Results Comparison}")
print("\\label{tab:comparison}")
print("\\end{table*}")

\begin{table*}[htbp]
\centering
\small
\setlength{\tabcolsep}{4pt}
\begin{tabular}{l|ccc|ccc|ccc|ccc}
\toprule
\multirow{2}{*}{Method} & \multicolumn{3}{c|}{3\#Step} & \multicolumn{3}{c|}{6\#Step} & \multicolumn{3}{c|}{12\#Step} & \multicolumn{3}{c}{Avg} \\
\cmidrule(lr){2-4} \cmidrule(lr){5-7} \cmidrule(lr){8-10} \cmidrule(lr){11-13}
& MAE & RMSE & MAPE & MAE & RMSE & MAPE & MAE & RMSE & MAPE & MAE & RMSE & MAPE \\
\midrule
retrain-st-sd & 23.86 & 35.38 & 17.17 & 25.59 & 38.16 & 18.74 & 29.09 & 43.57 & 22.20 & 26.33 & 38.54 & 19.05 \\
\midrule
pretrain-st-sd & 25.14 & 36.17 & 19.41 & 26.53 & 38.85 & 20.40 & 30.00 & 44.94 & 23.51 & 26.95 & 39.47 & 20.85 \\
\midrule
online-st-nn-sd & 25.98 & 38.71 & 17.42 & 27.72 & 41.54 & 19.33 & 34.25 & 47.61 & 24.46 & 30.76 & 42.06 & 19.99 \\
\midrule
online-st-an-sd & 25.19 & 36.19 & 20.79 & 26.75 & 38.92 & 22.35 & 30.31 & 44.61 & 26.23 & 29.01 & 39.40 & 22.81 \\
\midrule
trafficstream & 24.40 & 35.76 & 20.28 & 26.10 & 38.65 & 21.91 & 29.98 & 44.70 

<>:67: SyntaxWarning: invalid escape sequence '\#'
<>:67: SyntaxWarning: invalid escape sequence '\#'
/tmp/ipykernel_2184644/754085599.py:67: SyntaxWarning: invalid escape sequence '\#'
  print("\\multirow{2}{*}{Method} & \\multicolumn{3}{c|}{3\#Step} & \\multicolumn{3}{c|}{6\#Step} & \\multicolumn{3}{c|}{12\#Step} & \\multicolumn{3}{c}{Avg} \\\\")


In [1]:
import os
import re
import numpy as np

# 初始化一个字典来存储所有方法的结果
all_results = {}

# 首先收集所有方法的数据
for method in ["retrain_st_sd","pretrain_st_sd","online_st_nn_sd", "online_st_an_sd", "trafficstream", "stkec", "eac", "pecpm","dlf","team"]:
    base_dir = f"./log/GLA/{method}"
    directories = [f"{base_dir}-42"]
    
    metric_pattern = re.compile(r"(\d+|Avg)\s+(MAE|RMSE|MAPE)\s+([\d\.\s]+)")
    total_time_pattern = re.compile(r"total time:\s+([\d\.]+)")
    
    metrics = {
        "3": {"MAE": [], "RMSE": [], "MAPE": []},
        "6": {"MAE": [], "RMSE": [], "MAPE": []},
        "12": {"MAE": [], "RMSE": [], "MAPE": []},
        "Avg": {"MAE": [], "RMSE": [], "MAPE": []}
    }
    
    total_times = []
    
    def parse_log_file(file_path):
        with open(file_path, "r") as file:
            lines = file.readlines()[-22:] if method != 'pretrain_st_pems' else file.readlines()[-15:]
            for line in lines:
                match = metric_pattern.search(line)
                total_time_match = total_time_pattern.search(line)
                if match:
                    key, metric_type, values = match.groups()
                    values_list = list(map(float, values.split()))
                    if key in metrics:
                        metrics[key][metric_type].append(values_list[-1])
                if total_time_match:
                    total_times.append(float(total_time_match.group(1)))
    
    for directory in directories:
        log_file_path = os.path.join(directory, f"{method}.log")
        if os.path.exists(log_file_path):
            parse_log_file(log_file_path)
    
    def compute_statistics(metric_data):
        metric_array = np.array(metric_data)
        return np.mean(metric_array), np.std(metric_array)
    
    # 存储每个方法的结果
    method_results = []
    for key in ["3", "6", "12", "Avg"]:
        for metric_type in ["MAE", "RMSE", "MAPE"]:
            mean, std = compute_statistics(metrics[key][metric_type])
            method_results.append((mean, std))
    
    time_mean, time_std = compute_statistics(total_times)
    method_results.append((time_mean, time_std))
    
    all_results[method] = method_results

# 生成一个大的LaTeX表格
print("\\begin{table*}[htbp]")
print("\\centering")
print("\\small")  # 使用小号字体使表格更紧凑
print("\\setlength{\\tabcolsep}{4pt}")  # 减小列间距
print("\\begin{tabular}{l|ccc|ccc|ccc|ccc}")
print("\\toprule")  # 使用 booktabs 包的粗线
print("\\multirow{2}{*}{Method} & \\multicolumn{3}{c|}{3\#Step} & \\multicolumn{3}{c|}{6\#Step} & \\multicolumn{3}{c|}{12\#Step} & \\multicolumn{3}{c}{Avg} \\\\")
print("\\cmidrule(lr){2-4} \\cmidrule(lr){5-7} \\cmidrule(lr){8-10} \\cmidrule(lr){11-13}")  # 使用 booktabs 的中等粗细线
print("& MAE & RMSE & MAPE & MAE & RMSE & MAPE & MAE & RMSE & MAPE & MAE & RMSE & MAPE \\\\")
print("\\midrule")  # 使用 booktabs 的中等粗细线

# 为每个方法生成数据行
for method, results in all_results.items():
    row_data = [f"{mean:.2f}" for mean,_ in results[:-1]]  # 去掉最后一个时间数据
    print(f"{method.replace('_', '-')} & {' & '.join(row_data)} \\\\")
    print("\\midrule")  # 使用 booktabs 的中等粗细线

print("\\bottomrule")  # 使用 booktabs 的粗线
print("\\end{tabular}")
print("\\caption{Experimental Results Comparison}")
print("\\label{tab:comparison}")
print("\\end{table*}")

\begin{table*}[htbp]
\centering
\small
\setlength{\tabcolsep}{4pt}
\begin{tabular}{l|ccc|ccc|ccc|ccc}
\toprule
\multirow{2}{*}{Method} & \multicolumn{3}{c|}{3\#Step} & \multicolumn{3}{c|}{6\#Step} & \multicolumn{3}{c|}{12\#Step} & \multicolumn{3}{c}{Avg} \\
\cmidrule(lr){2-4} \cmidrule(lr){5-7} \cmidrule(lr){8-10} \cmidrule(lr){11-13}
& MAE & RMSE & MAPE & MAE & RMSE & MAPE & MAE & RMSE & MAPE & MAE & RMSE & MAPE \\
\midrule
retrain-st-sd & 27.17 & 38.64 & 15.65 & 29.29 & 42.09 & 16.88 & 33.37 & 48.48 & 19.66 & 33.89 & 42.45 & 17.15 \\
\midrule
pretrain-st-sd & 32.00 & 38.37 & 15.67 & 34.03 & 42.09 & 17.55 & 38.02 & 49.16 & 21.53 & 34.35 & 42.55 & 17.94 \\
\midrule
online-st-nn-sd & 28.00 & 39.60 & 19.39 & 31.03 & 44.00 & 22.50 & 41.70 & 52.15 & 28.57 & 37.08 & 44.52 & 22.98 \\
\midrule
online-st-an-sd & 26.89 & 38.25 & 15.83 & 29.16 & 41.90 & 17.39 & 33.56 & 48.69 & 20.96 & 33.79 & 42.31 & 17.78 \\
\midrule
trafficstream & 27.34 & 39.37 & 14.60 & 29.75 & 43.06 & 16.53 & 34.30 & 49.90 

<>:67: SyntaxWarning: invalid escape sequence '\#'
<>:67: SyntaxWarning: invalid escape sequence '\#'
/tmp/ipykernel_2686259/2669915334.py:67: SyntaxWarning: invalid escape sequence '\#'
  print("\\multirow{2}{*}{Method} & \\multicolumn{3}{c|}{3\#Step} & \\multicolumn{3}{c|}{6\#Step} & \\multicolumn{3}{c|}{12\#Step} & \\multicolumn{3}{c}{Avg} \\\\")
/home/bd2/miniconda3/envs/ANATS/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/bd2/miniconda3/envs/ANATS/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/bd2/miniconda3/envs/ANATS/lib/python3.12/site-packages/numpy/_core/_methods.py:223: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/bd2/miniconda3/envs/ANATS/lib/python3.12/site-packages/numpy/_core/_method

In [ ]:
import os
import re
import numpy as np

# 初始化一个字典来存储所有方法的结果
all_results = {}

# 首先收集所有方法的数据
for method in ["retrain_st_sd","pretrain_st_sd","online_st_nn_sd", "online_st_an_sd", "trafficstream", "stkec", "eac", "pecpm","dlf","team"]:
    base_dir = f"./log/CA/{method}"
    directories = [f"{base_dir}-42"]
    
    metric_pattern = re.compile(r"(\d+|Avg)\s+(MAE|RMSE|MAPE)\s+([\d\.\s]+)")
    total_time_pattern = re.compile(r"total time:\s+([\d\.]+)")
    
    metrics = {
        "3": {"MAE": [], "RMSE": [], "MAPE": []},
        "6": {"MAE": [], "RMSE": [], "MAPE": []},
        "12": {"MAE": [], "RMSE": [], "MAPE": []},
        "Avg": {"MAE": [], "RMSE": [], "MAPE": []}
    }
    
    total_times = []
    
    def parse_log_file(file_path):
        with open(file_path, "r") as file:
            lines = file.readlines()[-22:] if method != 'pretrain_st_pems' else file.readlines()[-15:]
            for line in lines:
                match = metric_pattern.search(line)
                total_time_match = total_time_pattern.search(line)
                if match:
                    key, metric_type, values = match.groups()
                    values_list = list(map(float, values.split()))
                    if key in metrics:
                        metrics[key][metric_type].append(values_list[-1])
                if total_time_match:
                    total_times.append(float(total_time_match.group(1)))
    
    for directory in directories:
        log_file_path = os.path.join(directory, f"{method}.log")
        if os.path.exists(log_file_path):
            parse_log_file(log_file_path)
    
    def compute_statistics(metric_data):
        metric_array = np.array(metric_data)
        return np.mean(metric_array), np.std(metric_array)
    
    # 存储每个方法的结果
    method_results = []
    for key in ["3", "6", "12", "Avg"]:
        for metric_type in ["MAE", "RMSE", "MAPE"]:
            mean, std = compute_statistics(metrics[key][metric_type])
            method_results.append((mean, std))
    
    time_mean, time_std = compute_statistics(total_times)
    method_results.append((time_mean, time_std))
    
    all_results[method] = method_results

# 生成一个大的LaTeX表格
print("\\begin{table*}[htbp]")
print("\\centering")
print("\\small")  # 使用小号字体使表格更紧凑
print("\\setlength{\\tabcolsep}{4pt}")  # 减小列间距
print("\\begin{tabular}{l|ccc|ccc|ccc|ccc}")
print("\\toprule")  # 使用 booktabs 包的粗线
print("\\multirow{2}{*}{Method} & \\multicolumn{3}{c|}{3\#Step} & \\multicolumn{3}{c|}{6\#Step} & \\multicolumn{3}{c|}{12\#Step} & \\multicolumn{3}{c}{Avg} \\\\")
print("\\cmidrule(lr){2-4} \\cmidrule(lr){5-7} \\cmidrule(lr){8-10} \\cmidrule(lr){11-13}")  # 使用 booktabs 的中等粗细线
print("& MAE & RMSE & MAPE & MAE & RMSE & MAPE & MAE & RMSE & MAPE & MAE & RMSE & MAPE \\\\")
print("\\midrule")  # 使用 booktabs 的中等粗细线

# 为每个方法生成数据行
for method, results in all_results.items():
    row_data = [f"{mean:.2f}" for mean,_ in results[:-1]]  # 去掉最后一个时间数据
    print(f"{method.replace('_', '-')} & {' & '.join(row_data)} \\\\")
    print("\\midrule")  # 使用 booktabs 的中等粗细线

print("\\bottomrule")  # 使用 booktabs 的粗线
print("\\end{tabular}")
print("\\caption{Experimental Results Comparison}")
print("\\label{tab:comparison}")
print("\\end{table*}")

\begin{table*}[htbp]
\centering
\small
\setlength{\tabcolsep}{4pt}
\begin{tabular}{l|ccc|ccc|ccc|ccc}
\toprule
\multirow{2}{*}{Method} & \multicolumn{3}{c|}{3\#Step} & \multicolumn{3}{c|}{6\#Step} & \multicolumn{3}{c|}{12\#Step} & \multicolumn{3}{c}{Avg} \\
\cmidrule(lr){2-4} \cmidrule(lr){5-7} \cmidrule(lr){8-10} \cmidrule(lr){11-13}
& MAE & RMSE & MAPE & MAE & RMSE & MAPE & MAE & RMSE & MAPE & MAE & RMSE & MAPE \\
\midrule
retrain-st-sd & 23.64 & 34.54 & 17.68 & 27.08 & 37.21 & 18.71 & 29.49 & 42.67 & 22.34 & 27.31 & 37.64 & 19.29 \\
\midrule
pretrain-st-sd & 26.65 & 35.62 & 19.62 & 27.91 & 38.21 & 20.39 & 31.10 & 44.42 & 22.99 & 28.28 & 38.87 & 20.78 \\
\midrule
online-st-nn-sd & 23.41 & 34.48 & 17.61 & 25.14 & 37.56 & 19.02 & 30.59 & 44.01 & 22.49 & 27.84 & 38.12 & 19.41 \\
\midrule
online-st-an-sd & 22.88 & 32.90 & 21.70 & 24.61 & 35.82 & 23.25 & 27.80 & 41.04 & 26.86 & 26.97 & 36.08 & 23.62 \\
\midrule
trafficstream & 26.99 & 35.70 & 23.16 & 28.26 & 38.38 & 24.77 & 30.72 & 43.33 

<>:67: SyntaxWarning: invalid escape sequence '\#'
<>:67: SyntaxWarning: invalid escape sequence '\#'
/tmp/ipykernel_1241284/813728148.py:67: SyntaxWarning: invalid escape sequence '\#'
  print("\\multirow{2}{*}{Method} & \\multicolumn{3}{c|}{3\#Step} & \\multicolumn{3}{c|}{6\#Step} & \\multicolumn{3}{c|}{12\#Step} & \\multicolumn{3}{c}{Avg} \\\\")
/home/bd2/miniconda3/envs/ANATS/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/bd2/miniconda3/envs/ANATS/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/bd2/miniconda3/envs/ANATS/lib/python3.12/site-packages/numpy/_core/_methods.py:223: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/bd2/miniconda3/envs/ANATS/lib/python3.12/site-packages/numpy/_core/_methods